In [ ]:
!pip install langchain
!pip install langchain-openai

!pip install serpapi
!pip install google-search-results
# load_toolsを利用するのに必要
!pip install langchain_community

In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['SERPAPI_API_KEY'] = userdata.get('SERPAPI_API_KEY')

# 3.3章

In [ ]:
!pip install langchainhub

In [ ]:
# プロンプトの取得
from langchain import hub

prompt = hub.pull("hwchase17/react")

print(prompt)

In [ ]:
# おみくじツール定義（再掲）

# おみくじ関数

import random
from datetime import datetime

def get_fortune(date_string):
    # 日付文字列を解析
    try:
        date = datetime.strptime(date_string, "%m月%d日")
    except ValueError:
        return "無効な日付形式です。'X月X日'の形式で入力してください。"

    # 運勢のリスト
    fortunes = [
        "大吉", "中吉", "小吉", "吉", "末吉", "凶", "大凶"
    ]

    # 運勢の重み付け（大吉と大凶の確率を低くする）
    weights = [1, 3, 3, 4, 3, 2, 1]

    # 日付に基づいてシードを設定（同じ日付なら同じ運勢を返す）
    random.seed(date.month * 100 + date.day)

    # 運勢をランダムに選択
    fortune = random.choices(fortunes, weights=weights)[0]

    return f"{date_string}の運勢は【{fortune}】です。"

# ツール作成

from langchain.tools import BaseTool

class Get_fortune(BaseTool):
    name = 'Get_fortune'
    description = (
        "特定の日付の運勢を占う。インプットは  'date_string'です。'date_string' は、占いを行う日付で、mm月dd日 という形式です。「1月1日」のように入力し、「'1月1日'」のように余計な文字列を付けてはいけません。"
    )

    def _run(self, date_string) -> str:
        return get_fortune(date_string)


    async def _arun(self, query: str) -> str:
        raise NotImplementedError("does not support async")

In [ ]:
# 日付ツール定義（再掲）

# 日付取得関数

from datetime import timedelta
from zoneinfo import ZoneInfo


def get_date(date):
    date_now = datetime.now(ZoneInfo("Asia/Tokyo"))
    if ("今日" in date):
        return date_now.strftime('%m月%d日')
    elif ("明日" in date):
        return (date_now + datetime.timedelta(days=1)).strftime('%m月%d日')
    elif ("明後日" in date):
        return (date_now + datetime.timedelta(days=2)).strftime('%m月%d日')
    else:
        return "サポートしていません"

class Get_date(BaseTool):
    name = 'Get_date'
    description = (
        "今日の日付を取得する。インプットは 'date'です。'date' は、日付を取得する対象の日で、'今日', '明日', '明後日' という3種類の文字列から指定します。「今日」のように入力し、「'今日'」のように余計な文字列を付けてはいけません。"
    )

    def _run(self, date) -> str:
        return get_date(date)

    async def _arun(self, query: str) -> str:
        raise NotImplementedError("does not support async")

In [ ]:
# エージェントの作成

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.agents import AgentExecutor, create_react_agent

model = ChatOpenAI(model="gpt-4o-mini")

tools = [Get_date(), Get_fortune()]
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


response = agent_executor.invoke({"input": [HumanMessage(content="今日の運勢を教えてください。")]})

print(response)

In [ ]:
# 検索できるエージェント作成

from langchain.agents import AgentExecutor, create_react_agent, load_tools

model = ChatOpenAI(model="gpt-4o-mini")
tools = load_tools(["serpapi"], llm=model)
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


response = agent_executor.invoke({"input": [HumanMessage(content="株式会社Elithの住所を教えてください。最新の公式情報として公開されているものを教えてください。")]})

print(response)